In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from stable_baselines3 import PPO
import sys
import os

# Import the environment from the training notebook
sys.path.insert(0, os.path.abspath('.'))

np.random.seed(42)

## Load Trained Model

In [ ]:
# Load the trained model
model_path = os.path.join('..', 'models', 'trained_rl_model')
model = PPO.load(model_path)

print(f"Model loaded successfully from {model_path}")
print(f"Model type: {type(model)}")

## Evaluation Metrics

## Run Evaluation

In [ ]:
# Need to reimport the environment class
# For this example, we'll create a simplified version

from gym import spaces
import gym

class FactoryEnergyEnv(gym.Env):
    """Simplified environment for evaluation."""
    
    def __init__(self, max_steps=3600):
        self.max_steps = max_steps
        self.current_step = 0
        self.action_space = spaces.Discrete(8)
        self.observation_space = spaces.Box(low=0, high=100, shape=(10,), dtype=np.float32)
        self.state = None
    
    def reset(self):
        self.current_step = 0
        self.state = np.array([
            np.random.uniform(5, 15), np.random.uniform(2, 8), np.random.uniform(5, 12),
            0, np.random.uniform(0.3, 0.9), np.random.uniform(0.2, 0.8),
            np.random.uniform(0.3, 0.9), 0, 0, 0
        ], dtype=np.float32)
        self.state[3] = self.state[0] + self.state[1] + self.state[2]
        return self.state
    
    def step(self, action):
        self.current_step += 1
        load_adj = (action - 3.5) * 0.05
        self.state[4] = np.clip(self.state[4] + load_adj, 0.1, 1.0)
        self.state[5] = np.clip(self.state[5] - load_adj * 0.5, 0.1, 1.0)
        self.state[6] = np.clip(self.state[6] - load_adj * 0.7, 0.1, 1.0)
        self.state[0] = 15 * self.state[4]
        self.state[1] = 8 * self.state[5]
        self.state[2] = 12 * self.state[6]
        self.state[3] = self.state[0] + self.state[1] + self.state[2]
        self.state += np.random.normal(0, 0.5, size=self.state.shape)
        self.state = np.clip(self.state, 0, 100).astype(np.float32)
        self.state[7] = self.current_step
        self.state[8] += self.state[3] * 0.1 / 3600
        reward = -self.state[3] / 40 - 0.1 * abs(sum(self.state[4:7]) - 1.5)
        self.state[9] = reward
        done = self.current_step >= self.max_steps
        info = {'total_power': float(self.state[3]), 'energy_consumed': float(self.state[8])}
        return self.state, reward, done, info

# Create environment
eval_env = FactoryEnergyEnv(max_steps=1000)

# Evaluate
metrics, rewards, energies = evaluate_policy(model, eval_env, num_episodes=5)

print("\nEvaluation Results:")
print(f"  Mean Reward: {metrics['mean_reward']:.4f} ± {metrics['std_reward']:.4f}")
print(f"  Mean Energy: {metrics['mean_energy']:.4f} ± {metrics['std_energy']:.4f} kWh")
print(f"  Mean Power: {metrics['mean_power']:.2f} kW")
print(f"  Mean Episode Length: {metrics['mean_episode_length']:.0f} steps")

## Visualization of Results

In [ ]:
# Visualize power consumption across episodes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(len(rewards)), rewards, color='steelblue')
axes[0].set_title('Episode Rewards')
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Total Reward')
axes[0].axhline(y=np.mean(rewards), color='r', linestyle='--', label='Mean')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].bar(range(len(energies)), energies, color='green')
axes[1].set_title('Episode Energy Consumption')
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Energy (kWh)')
axes[1].axhline(y=np.mean(energies), color='r', linestyle='--', label='Mean')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Export Evaluation Report

In [ ]:
# Export evaluation results
report_path = os.path.join('..', 'docs', 'validation_report.csv')
os.makedirs(os.path.dirname(report_path), exist_ok=True)

report_data = {
    'metric': list(metrics.keys()),
    'value': [metrics[k] if not isinstance(metrics[k], list) else str(metrics[k]) for k in metrics.keys()]
}

report_df = pd.DataFrame(report_data)
report_df.to_csv(report_path, index=False)
print(f"Report saved to {report_path}")